In [1]:
%env CUPY_ACCELERATORS = cub
import sys
import unittest

import dask.array
import dask.distributed as dd
import cupy
import xarray as xr

import src.geocat.comp.meteorology_gpu_reduction as geo_gpu_red
import src.geocat.comp.meteorology_gpu as geo_gpu
import src.geocat.comp.meteorology as geo

import numpy
import time
import pandas as pd

csvpath = "Test_meteorology.csv"

env: CUPY_ACCELERATORS=cub


In [2]:
#Test_dewtemp helper
HelperFunction = "_dewtemp"
print(HelperFunction)
allData = pd.DataFrame()
for i in range(1,8):
    ArraySize = 10**i
    print("Array size: ", ArraySize)
    for xp in ["numpy", "reduction","cupy"]:
        repeat = numpy.zeros([10,])
        repsize = 10
        for rep in range(0,repsize):
            t_def = numpy.random.uniform(low=-52,high=29.5,size=ArraySize)
            rh_def = numpy.random.uniform(low=41.7,high=90.5, size=ArraySize)
            tk = numpy.add(cupy.asarray(t_def),273.15)
            cupy.cuda.runtime.deviceSynchronize()
            time1 = time.time()
            if(xp == "cupy"):
                rh_def = cupy.asarray(rh_def)
                tk = cupy.asarray(tk)
                dt_2 = geo_gpu._dewtemp_gpu(tk, rh_def) - 273.15
                #validating GPU result with CPU results
                if not numpy.allclose(cupy.asnumpy(dt_2),geo._dewtemp(cupy.asnumpy(tk), cupy.asnumpy(rh_def)) - 273.15): print("FALSE")
            if(xp == "reduction"):
                rh_def = cupy.asarray(rh_def)
                tk = cupy.asarray(tk)
                dt_2 = cupy.add(geo_gpu_red._dewtemp_gpu_red(tk, rh_def),- 273.15)
                if not numpy.allclose(cupy.asnumpy(dt_2),geo._dewtemp(cupy.asnumpy(tk),cupy.asnumpy(rh_def)) - 273.15): print("FALSE")
            else:
                rh_def = cupy.asnumpy(rh_def)
                tk = cupy.asnumpy(tk)
                dt_2 = geo._dewtemp(tk, rh_def) - 273.15
            cupy.cuda.runtime.deviceSynchronize()
            time2 = time.time()
            repeat[rep] = time2-time1
        #save times
        data = {'HelperFunction': numpy.repeat(HelperFunction, repsize),
                'Approach': numpy.repeat(xp , repsize),
                'ArraySize': numpy.repeat(ArraySize , repsize),
                'iteration' : numpy.arange(1,repsize+1),
                'Runtime(s)': repeat}
        new = pd.DataFrame(data)
        allData = pd.concat([allData,new], ignore_index=True)
        print(xp,numpy.mean(repeat), "seconds") 
#try:
    #previous = pd.read_csv(csvpath)
    #previous = pd.concat([previous,allData])
#except FileNotFoundError:
    #previous = allData
#previous.to_csv(csvpath, index=False)

_dewtemp
Array size:  10
numpy 6.189346313476563e-05 seconds
reduction 0.0012747764587402344 seconds
<class 'cupy._core.core.ndarray'>
[49.32233936 81.41726661 83.87788227 61.58363169 51.27835524 53.41160762
 84.9230248  57.40373807 51.75391395 65.99583488]
<class 'cupy._core.core.ndarray'>
[41.9057188  47.49735747 44.79568675 46.53999391 64.34788045 66.21608451
 53.33385172 55.83902695 65.21557098 61.55845478]
<class 'cupy._core.core.ndarray'>
[89.94536027 63.90631017 47.39702687 79.59237694 53.06310534 66.84377611
 66.37801488 56.64493992 78.37575159 42.92564719]
<class 'cupy._core.core.ndarray'>
[90.26637416 79.16894227 86.43773035 87.80864185 63.37512625 74.04354496
 48.06896872 49.52200027 44.31129244 48.63246723]
<class 'cupy._core.core.ndarray'>
[66.08022159 58.12995354 88.27830456 55.52423262 44.39516466 80.39399128
 57.90404937 84.35724808 69.42618636 56.70082441]
<class 'cupy._core.core.ndarray'>
[43.75881655 43.41320845 83.46566536 64.48319279 48.66486294 76.9501072
 89.0403

In [3]:
#reimplemented test functions for GPU/CPU benchmarking 
#Test_dewtemp
def test_list_input(xp = cupy):
    tk = (xp.asarray(t_def) + 273.15).tolist()
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh_def) - 273.15, dt_2, 0.1)
    else:
        assert xp.allclose(geo.dewtemp(tk, rh_def) - 273.15, dt_2, 0.1)

def test_numpy_input(xp = cupy):
    tk = xp.asarray(t_def) + 273.15
    rh = xp.asarray(rh_def)
    if(xp.__name__ == "cupy"):
        assert xp.allclose(geo_gpu.dewtemp_gpu(tk, rh) - 273.15, dt_2, 0.1)
    else:
        assert xp.allclose(geo.dewtemp(tk, rh) - 273.15, dt_2, 0.1)

In [1]:
import dask.array as da
t_def = da.random.uniform(low=-52,high=29.5,size=10**10)
rh_def = da.random.uniform(low=41.7,high=90.5, size=10**10)
tk = da.add(da.asarray(t_def),273.15)

In [ ]:
import cupy as cp
cp.asarray(tk)

In [ ]:
geo._dewtemp(tk,rh_def)

In [ ]:
tk = cupy.asarray(tk)
rh_def = cupy.asarray(rh_def)
geo_gpu._dewtemp_gpu(tk,rh_def)

In [ ]:
tk = cupy.asarray(tk)
rh_def = cupy.asarray(rh_def)
geo_gpu_red._dewtemp_gpu_red(tk,rh_def)

In [4]:
import cupy as cp
import numpy as np
tk = cp.asnumpy(tk)
time1 = time.time()
b = tk*0.57
time2 = time.time()
print(b)
time2-time1

[160.69581962 164.94031872 144.69203798 ... 164.46251337 138.25559474
 155.46056802]


0.03907608985900879

In [7]:
import cupy as cp
cupy.cuda.runtime.deviceSynchronize()
time1 = time.time()
tk = cp.asarray(tk)
c = cp.multiply(tk,0.57)
cupy.cuda.runtime.deviceSynchronize()
time2 = time.time()
print(c)
time2-time1

[160.69581962 164.94031872 144.69203798 ... 164.46251337 138.25559474
 155.46056802]


0.0004086494445800781

In [6]:
import cupy as cp
cupy.cuda.runtime.deviceSynchronize()
time1 = time.time()
tk = cp.asarray(tk)
d = tk*0.57
cupy.cuda.runtime.deviceSynchronize()
time2 = time.time()
print(d)
time2-time1

[160.69581962 164.94031872 144.69203798 ... 164.46251337 138.25559474
 155.46056802]


0.0004398822784423828

In [ ]:
tk = cupy.add(tk,-273)

In [ ]:
tk

In [ ]:
-.57* tk